# 1D 컨브넷
- Conv1D 층을 사용하여 구현, Conv2D와 인터페이스가 비슷함
- (samples, timestep, features) 크기의 3D 텐서를 입력 받고 비슷한 형태의 3D 텐서를 반환함
- 합성곱 윈도는 시간 측의 1D 윈도이며, 입력텐서의 두 번째 축임 

In [2]:
from keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [5]:
max_features = 10000
maxlen = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), len(x_test))

25000 25000


In [6]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [7]:
x_train.shape

(25000, 500)

In [8]:
x_test.shape

(25000, 500)

In [9]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

In [23]:
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=5))
model.add(layers.Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(layers.GlobalMaxPooling1D()) # Flatten 함수 대신 사용
model.add(layers.Dense(units=1))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_16 (Conv1D)          (None, 494, 32)           28704     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 98, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                

In [30]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics='acc')

model.fit(x_train, y_train, epochs = 10,
         batch_size=128, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 39s 236ms/step - loss: 0.2707 - acc: 0.9275 - val_loss: 0.5500 - val_acc: 0.8622
Epoch 2/10
157/157 [==============================] - 38s 240ms/step - loss: 0.1886 - acc: 0.9567 - val_loss: 0.7439 - val_acc: 0.8542
Epoch 3/10
157/157 [==============================] - 36s 227ms/step - loss: 0.1431 - acc: 0.9744 - val_loss: 0.6534 - val_acc: 0.8794
Epoch 4/10
157/157 [==============================] - 37s 237ms/step - loss: 0.1091 - acc: 0.9869 - val_loss: 1.1876 - val_acc: 0.8388
Epoch 5/10
157/157 [==============================] - 37s 233ms/step - loss: 0.0935 - acc: 0.9914 - val_loss: 1.0221 - val_acc: 0.8660
Epoch 6/10
157/157 [==============================] - 35s 223ms/step - loss: 0.0823 - acc: 0.9931 - val_loss: 0.9396 - val_acc: 0.8744
Epoch 7/10
157/157 [==============================] - 34s 219ms/step - loss: 0.0786 - acc: 0.9933 - val_loss: 1.3047 - val_acc: 0.8506
Epoch 8/10
157/157 [==============================] - 3

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 정리
- 검증 정확도는 LSRM(약 0.89)보다 조금 낮지만, CPU와 GPU에서 더 빠르게 실행됨
- 아는 단어 수준의 감성 분류 작업에 순환 네트워크를 대신하여 빠르고 경제적인 1D 컨브넷을 사용할 수 있음